In [2]:
import os

os.chdir("../")

In [3]:
%pwd

'c:\\Users\\rupan\\Documents\\Projects\\Deep Learning\\Chicken-Disease-Classification'

In [4]:
import tensorflow as tf


model = tf.keras.models.load_model("artifacts/training/model.keras")


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: list
    

In [6]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        training_config = self.config.training
        data_ingestion_config = self.config.data_ingestion
        eval_config = EvaluationConfig(
            path_of_model=Path(training_config.training_model_path),
            training_data=os.path.join(data_ingestion_config.unzip_dir, "Chicken-fecal-images"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )


        return eval_config

In [7]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"accuracy": self.score[1], "loss": self.score[0]}
        save_json(path=Path("scores.json"), data=scores)

[2024-06-28 16:09:57,008: INFO : common: yaml file: config\config.yaml loaded successfully]
[2024-06-28 16:09:57,016: INFO : common: yaml file: params.yaml loaded successfully]
[2024-06-28 16:09:57,018: INFO : common: created directory at: artifacts]
Found 116 images belonging to 2 classes.


c:\Users\rupan\Documents\Projects\Deep Learning\Chicken-Disease-Classification\chicken_venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 63s 8s/step - accuracy: 0.9011 - loss: 0.4793
[2024-06-28 16:11:00,734: INFO : common: json file saved at scores.json]


In [ ]:
def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

In [9]:
def _valid_generator(config):

    datagenerator_kwargs = dict(
        rescale = 1./255,
        validation_split=0.30
    )

    dataflow_kwargs = dict(
        target_size=config.params_image_size[:-1],
        batch_size=config.params_batch_size,
        interpolation="bilinear"
    )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
    )

    valid_generator = valid_datagenerator.flow_from_directory(
        directory=config.training_data,
        subset="validation",
        shuffle=False,
        **dataflow_kwargs
    )

    return valid_generator

In [ ]:
config = ConfigurationManager()
val_config = config.get_validation_config()
evaluation = Evaluation(val_config)

In [ ]:
model = load_model(val_config.path_of_model)


In [ ]:
datagenerator_kwargs = dict(
        rescale = 1./255,
        validation_split=0.30
    )

dataflow_kwargs = dict(
        target_size=val_config.params_image_size[:-1],
        batch_size=val_config.params_batch_size,
        interpolation="bilinear"
    )

In [ ]:
datagenerator_kwargs = dict(
        rescale = 1./255,
        validation_split=0.30
    )

dataflow_kwargs = dict(
        target_size=val_config.params_image_size[:-1],
        batch_size=val_config.params_batch_size,
        interpolation="bilinear"
    )

valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
    )

valid_datagenerator.flow_from_directory(
        directory=val_config.training_data,
        subset="validation",
        shuffle=False,
        **dataflow_kwargs
    )

In [ ]:
valid_gen = _valid_generator(val_config)
model.evaluate(valid_gen)


In [ ]:
for batch in valid_gen:
    if isinstance(batch, tuple) and len(batch) == 2:
        x, y = batch
        print(f"x shape: {x.shape}, y shape: {y.shape}")
    else:
        print("Unexpected batch format or content is None")
    break  


In [ ]:
batch = next(iter(valid_gen))
print(batch)



In [ ]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e